# How to use cross validation

In [1]:
import pandas  as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.optimization as opt
from biogeme.expressions import Beta, DefineVariable
import seaborn as sns
import matplotlib.pyplot as plt

**Import Swissmetro data**

In [2]:
pandas = pd.read_csv("../../Data/8-DiscreteChoiceModels/swissmetro.dat",sep='\t')
database = db.Database("data/swissmetro", pandas)

**Use collumn names as variables**

In [3]:
globals().update(database.variables)

**Exclude some unwanted entries**

In [4]:
exclude = (( PURPOSE != 1 ) * ( PURPOSE != 3 ) + ( CHOICE == 0 )) > 0

database.remove(exclude)

**Define some dummy variables**

In [5]:
SM_COST = SM_CO * ( GA == 0 )
TRAIN_COST = TRAIN_CO * ( GA == 0 )

CAR_AV_SP = DefineVariable ('CAR_AV_SP', CAR_AV * ( SP !=0 ), database)
TRAIN_AV_SP = DefineVariable ('TRAIN_AV_SP', TRAIN_AV * ( SP != 0 ), database)

**Rescale some data**

In [6]:
TRAIN_TT_SCALED   = DefineVariable('TRAIN_TT_SCALED',   TRAIN_TT / 100.0, database)
TRAIN_COST_SCALED = DefineVariable('TRAIN_COST_SCALED', TRAIN_COST / 100, database)
SM_TT_SCALED      = DefineVariable('SM_TT_SCALED',      SM_TT / 100.0   , database)
SM_COST_SCALED    = DefineVariable('SM_COST_SCALED',    SM_COST / 100   , database)
CAR_TT_SCALED     = DefineVariable('CAR_TT_SCALED',     CAR_TT / 100    , database)
CAR_CO_SCALED     = DefineVariable('CAR_CO_SCALED',     CAR_CO / 100    , database)

**Create parameters to be estimated**

In [7]:
ASC_CAR = Beta('ASC_CAR',0,None ,None ,0)
ASC_TRAIN = Beta('ASC_TRAIN',0,None ,None ,0)
ASC_SM = Beta('ASC_SM',0,None ,None ,1)
B_TIME = Beta('B_TIME',0,None ,None ,0)
B_COST = Beta('B_COST',0,None ,None ,0)

**Define the utility functions**

In [8]:
V1 = ASC_TRAIN + \
     B_TIME * TRAIN_TT_SCALED + \
     B_COST * TRAIN_COST_SCALED
V2 = ASC_SM + \
     B_TIME * SM_TT_SCALED + \
     B_COST * SM_COST_SCALED
V3 = ASC_CAR + \
     B_TIME * CAR_TT_SCALED + \
     B_COST * CAR_CO_SCALED

**Associate utility functions with alternatives and associate availability of alternatives**

In [9]:
V = {1: V1,
     2: V2,
     3: V3}

av = {1: TRAIN_AV_SP,
      2: SM_AV,
      3: CAR_AV_SP}

**Define the model**

In [10]:
logprob = models.loglogit(V, av, CHOICE)

**Define the Biogeme object**

In [11]:
biogeme  = bio.BIOGEME(database, logprob)

biogeme.modelName = "swissmetro_logit_validation"

**Estimate the model**

1. A `.html` can be generated with a report of the results and can be opened with a browser
2. A `.pickle` file can also be generaetd with a snapshot with the results. This file can then be used in other scripts

In [12]:
biogeme.generateHtml = True
biogeme.generatePickle = False

results = biogeme.estimate()

print(f"HTML file:    {results.data.htmlFileName}")
print(f"Pickle file:  {results.data.pickleFileName }")

HTML file:    swissmetro_logit_validation~26.html
Pickle file:  None


In [13]:
gs = results.getGeneralStatistics()

for k,v in gs.items():
    print("{}= {}".format(k.ljust(45),v[0]))

Number of estimated parameters               = 4
Sample size                                  = 6768
Excluded observations                        = 3960
Init log likelihood                          = -5331.252006916162
Final log likelihood                         = -5331.252006916162
Likelihood ratio test for the init. model    = -0.0
Rho-square for the init. model               = 0.0
Rho-square-bar for the init. model           = -0.0007502928007925291
Akaike Information Criterion                 = 10670.504013832324
Bayesian Information Criterion               = 10697.783857437467
Final gradient norm                          = 0.0006288284529603027
Nbr of threads                               = 8


In [14]:
validationData = database.split(slices=5)

validation_results = biogeme.validate(results, validationData)

for slice_i in validation_results:
    print(
        f'Log likelihood for {slice_i.shape[0]} validation data: '
        f'{slice_i["Loglikelihood"].sum()}'
    )

Log likelihood for 1354 validation data: -1053.8922182264362
Log likelihood for 1354 validation data: -1041.8803015559752
Log likelihood for 1354 validation data: -1123.1198254113378
Log likelihood for 1353 validation data: -1070.429949604534
Log likelihood for 1353 validation data: -1041.9297121178788
